<a href="https://colab.research.google.com/github/ohhongseok/ML-DeepLearning/blob/main/4/4-1/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로지스틱 회귀로 다중분류 수행하기

 - LogisticRegression 을 활용하여 다중분류 진행
  > 모델 특징 <br>
  1. 반복적인 알고리즘을 하며, `max_iter` 매개변수를 지정하여 사용, 기본은 100 이다.<br>
  2. 릿지 회귀와 같이 계수의 제곱을 규제한다. 여기에서는 L2 규제라고 한다.<br>
  3. 로지스틱 회귀에서 규제를 하는 매개변수는 C 이며, 기본값은 1 이다.<BR>
  4. 릿지와 다르게 로지스틱회귀에서 규제는 값이 커질수록 ***규제가 작아진다.***

In [ ]:
# 데이터 불러오기 및 test train 데이터 나누기
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

fish = pd.read_csv('https://bit.ly/fish_csv_data')

fish_input = fish[['Weight',	'Length',	'Diagonal',	'Height',	'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 정규화된 train input data
# 훈련세트의 통계값으로 테스트 세트를 변환해야 한다.
test_scaled = ss.transform(test_input) # 정규화된 test input data

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))
# 훈련세트와 테스트 세트에 대해 점수가 높고
# 과대적합, 과소적합이 아님

0.9327731092436975
0.925


In [ ]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [ ]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [ ]:
# 각 셈플별 클래스의 확률을 수치화
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))

# 총 5개의 샘플이 들어갔기에 5행이 출력,
# 다중회귀(7개의 클래스 있음) 임으로 7개의 열이 출력됨
# 첫번째 샘플의 경우 세번째 열의 데이터의 가능성이 제일 높음(0.841) -> perch

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [ ]:
#다중분류의 선형방정식 확인해 보기
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


# coef / intercept_ 의미 해석하기
1. coef_ (7,5) -> 총 5개의 특성 데이터('Weight',	'Length',	'Diagonal',	'Height',	'Width' 를 사용한다. 따라서 5임)
2. 총 7개의 z 값을 계산하며,(다중클래스는 클래스의 갯수만큼 z를 계산한다.) 이들중 가장 높은 값을 선정한다.
3. z 값은 소프트 맥스 함수를 활용하여 7개의 z값을 확률로 반환함.
  > 소프트맥스 함수<br>
  1. 시그모이드 함수는 하나의 선형방정식에 대해 0~1 사이의 값을 출력한다.
  2. 소프트맥스 함수는 n개의 선형방정식의 전체 합이 1이 되도록 만든다. 이러한 특징으로 인해 ***정규화된 지수함수*** 라고도 한다.

In [ ]:
decision = lr.decision_function(test_scaled[:5]) #z의 값 출력
# 다중 클래스의 경우 클래스의 개수만큼 z값을 계산 => 총 7개 따아서 z값도 7개
print(np.round(decision,decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [ ]:
from scipy.special import softmax
proba = softmax(decision,axis=1)
print(np.round(proba,decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


# 이번장 정리
 - 로지스틱 회귀 : 선형방정식을 사용한 분류 알고리즘으로 시그모이드(이진분류), 소프트맥스(다중분류) 함수를 통해 클래스 확률 출력
 - 다중분류는 타깃 클래스가 2개 이상인 분류 문제 로지스틱회귀는 소프트맥스 함수를 사용하여 클래스 예측
 - 시그모이드 함수 : 선형방정식의 출력을 0~1로 압축하고, 이진분류를 위해 사용
 - 소프트맥스 함수 : 다중분류에서 여러 선형방정식 출력 결과를 정규화 해 1의 값을 가질 수 있도록 만듦
 